In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib  inline





def read_tdxlday_2csv(dirname, fname, targetDir):
    import os
    from struct import unpack, pack
    outfile=open(dirname + os.sep + fname, 'rb')
    buf = outfile.read()
    outfile.close()
    
    inputfile=open(targetDir + os.sep + fname + '.csv', 'w')
    num=len(buf)
    numrec=num/32
    b=0
    e=32
    line=''
    linename=u'date, open, high, low, close, amount, volume, str07' + '\n'
    
    inputfile.write(linename)
    for i in xrange(numrec):
        a=unpack('IIIIIfII', buf[b:e])
        #   结尾的"\" 表示续行
        line=str(a[0])+', ' +str(a[1]/100.0)+', ' +str(a[2]/100.0)+', ' +str(a[3]/100.00)+', '     +  \
             str(a[4]/100.0)+', ' +str(a[5]/1.0)+', ' +str(a[6]/100.0)+', ' +str(a[7])+'' +'\n'
        inputfile.write(line)
        b+=32
        e+=32
    inputfile.close()
        
        

#%%
def read_lday_csvFile(fname):
    import pandas as pd
    u'''read joinquant export csv file to dataframe
    param::
        fname
    return::
		4个对象的元组, (ohlc数据框, 代码, roc1序列,收益率序列)
        df, ticker, roc1, BnH
    example::
        fname= "d:/db/downloads/150195.csv"
        ohlc,ticker,roc1,BnH = read_lday_csvFile(fname)
    '''
#    fname = "d:/db/downloads/159902.csv"
    ohlc = pd.read_csv(fname, index_col=0, parse_dates=True)
#    ohlc_copy = ohlc.loc[:,:]
#    num_samples = len(ohlc)
##    ohlc.iloc[:,(0,1,2,3)].plot()
#    #df= randprice(240,20)
#    
#    n_samples = num_samples - 100  #1500
#    ohlc = ohlc_copy[-n_samples:]
    ohlc.columns = ohlc.columns.str.replace(u' open', 'O')
    ohlc.columns = ohlc.columns.str.replace(u' high', 'H')
    ohlc.columns = ohlc.columns.str.replace(u' low', 'L')
    ohlc.columns = ohlc.columns.str.replace(u' close', 'C')
    ohlc.columns = ohlc.columns.str.replace(u' amount', 'A')
    ohlc.columns = ohlc.columns.str.replace(u' volume', 'V')
    
    ohlc.index.name='tdx_DateTime'
    
    roc1 = ohlc.C.pct_change().fillna(value=0);  roc1 = roc1.rename('ROC1')
    bnh = (1.0+roc1).cumprod()
    bnh = bnh.rename('BnH')
    ticker = fname.split('/')[-1][0:8]
    ticker = ticker.upper()
    
    return ohlc, ticker, roc1, bnh
    


def HSL(vol, n):
    out = vol/talib.SUM(vol,n)
    # 处理nan: 前面的n-1个nan values
    for i in np.arange(n-1):
        out[i]=vol[i]/vol.cumsum()[i]
    return out
            

def DMA(p,f):
    y=[0]*len(p)
    for i in np.arange(len(p)):
        if i==0:
            y[i] = p[i]
        else:
            y[i] = f[i]*p[i] + (1-f[i])*y[i-1]
    return y

             

In [5]:
"""This cell defineds the plot_candles function"""

def plot_candles(pricing, title=None, volume_bars=False, color_function=None, technicals=None, savefig=False):
    """ Plots a candlestick chart using quantopian pricing data.
    
    Author: Daniel Treiman
    
    Args:
      pricing: A pandas dataframe with columns ['open_price', 'close_price', 'high', 'low', 'volume']
      title: An optional title for the chart
      volume_bars: If True, plots volume bars
      color_function: A function which, given a row index and price series, returns a candle color.
      technicals: A list of additional data series to add to the chart.  Must be the same length as pricing.
    """
    def default_color(index, open_price, close_price, low, high):
        return 'black' if open_price[index] > close_price[index] else 'white' # r g b  cyan black white
    
    color_function = color_function or default_color
    technicals = technicals or []
    open_price = pricing['O']
    close_price = pricing['C']
    low = pricing['L']
    high = pricing['H']
    oc_min = pd.concat([open_price, close_price], axis=1).min(axis=1)
    oc_max = pd.concat([open_price, close_price], axis=1).max(axis=1)
    
    if volume_bars:
        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, gridspec_kw={'height_ratios': [3,1]} )
    else:
        fig, ax1 = plt.subplots(1, 1)
    fig = plt.gcf()
    #fig.set_size_inches(18.5, 10.5)
    fig.set_size_inches(len(pricing)/10.0, 12)
    
    if title:
        ax1.set_title(title)
    x = np.arange(len(pricing))
    candle_colors = [color_function(i, open_price, close_price, low, high) for i in x]
#    candles = ax1.bar(x, oc_max-oc_min, bottom=oc_min, color=candle_colors, linewidth=0)
#    lines = ax1.vlines(x + 0.4, low, high, color=candle_colors, linewidth=1)
    candles = ax1.bar(x-0.4, oc_max-oc_min, bottom=oc_min, color=candle_colors, linewidth=0.2, edgecolor='black')
    lines = ax1.vlines(x,    oc_max, high, color=['black']*len(pricing), linewidth=1)
    lines = ax1.vlines(x,    low, oc_min,  color=['black']*len(pricing), linewidth=1)
    ax1.xaxis.grid(False)
    ax1.xaxis.set_tick_params(which='major', length=3.0, direction='in', top='off')
    # Assume minute frequency if first two bars are in the same day.
    frequency = 'minute' if (pricing.index[1] - pricing.index[0]).days == 0 else 'day'
    time_format = '%Y-%m-%d'
    if frequency == 'minute':
        time_format = '%H:%M'
    # Set X axis tick labels.
    x5 = np.arange(0,len(pricing), 5)
    dt_index5 = [pricing.index[i]  for i in x5]
    plt.xticks(x5, [date.strftime(time_format) for date in dt_index5], rotation=45)
    #plt.xticks(x, [date.strftime(time_format) for date in pricing.index], rotation='vertical')
    
    for indicator in technicals:
        ax1.plot(x, indicator, 'o-', linewidth=0.1, markersize=0.7, markeredgewidth=0.1) #带圆圈标记的实线
#     if dma61 in technicals: #.name = 'dma61': ax1.plot(x, indicator, linewidth=2)
#         ax1.plot(x, dma61, 'o-', color='green', linewidth=0.1, markersize=0.7, markeredgecolor='green', markeredgewidth=0.1) 
    
    if volume_bars:
        volume = pricing['V']
        volume_scale = None
        scaled_volume = volume
        if volume.max() > 1000000:
            volume_scale = 'M'
            scaled_volume = volume / 1000000
        elif volume.max() > 1000:
            volume_scale = 'K'
            scaled_volume = volume / 1000
        ax2.bar(x-0.4, scaled_volume, color=candle_colors, edgecolor='black')
        volume_title = 'Volume'
        if volume_scale:
            volume_title = 'Volume (%s)' % volume_scale
        ax2.set_title(volume_title)
        ax2.xaxis.grid(False)
        
    if savefig:
        fig.savefig('test2png.png', dpi=300)

In [ ]:
li1=np.arange(5)
li2=np.arange(8)
var=[li1, li2]
print li1, li2, var

li2 in var

# plt.xticks?
x5 = np.arange(0,len(ohlc), 5)
dt_index5 = [ohlc.index[i]  for i in x5]
# plt.xticks(x5, [date.strftime(time_format) for date in dt_index5], rotation=20)

#" 转换为csv"  调用示例:
# pathdir='/vipdoc/sh/lday'  #d:\new_pazq\Vipdoc\sz\lday\sz000032.day
pathdir='d:/new_pazq/Vipdoc/sz/lday'
targetDir= 'd:/temp/python_data_gupiao/lday'

#listfile=os.listdir(pathdir)
#for f in listfile:
#      day2csv_data(pathdir,f,targetDir)
        
fname='sz000911.day'

read_tdxlday_2csv(pathdir, fname, targetDir)

In [3]:
# 读取csv
fname='d:/temp/python_data_gupiao/lday/sz000911.day.csv'
#ohlc = pd.read_csv(fname, index_col=0, parse_dates=True)
ohlc,ticker,roc1,bnh = read_lday_csvFile(fname)
bnh = (1.0+roc1).cumprod();    bnh = bnh.rename('BnH')

print ohlc.C.head(), '\n', roc1.head(), bnh.head(6)
print len(ohlc), len(ticker), len(roc1), len(bnh)
print type(ohlc), type(ticker), type(roc1), type(bnh)

ohlc.columns

bnh.plot(title=ticker,legend=True, logy=True)

ohlc.C.plot(title=ticker,legend=True, logy=True)


import math

# math.log 仅支持标量运算
# np.log2?

#log2_C=pd.Series(np.log2(ohlc.C.values), index=ohlc.index)
#log2_C.plot(title=ticker+', Log2 scale', legend=True)

def pd_log2(x):
    return pd.Series(np.log2(x.values), index= x.index)
                     
log_bnh = pd_log2(bnh)
log_bnh[:40].plot()

log_bnh.plot()

plot_candles(ohlc.head(30), title=ticker+' day candles', volume_bars=False)

plot_candles(ohlc.tail(50), title=ticker+' day candles', volume_bars=True)

fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.bar?

fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
#fig.savefig('test2png.png', dpi=100)

Command mode short-cut keys:
c       to copy cell
x       to delete cell   
j       to select cell below 选择下方单元
k       to select cell above 选择上方单元
shift+j to extend select cell below 扩展选择下方单元
shift+k to extend select cell above 扩展选择上方单元
v       to paste cell above
shift-V to paste cell below
shift-M to merge selected cells, or merge current cell with cell below

#魔法命令pycat : cat(显示) 一个py脚本文件到输出页面里
%pycat dict_indexcode.py
%pycat dict_indexcode.py

?%pycat #显示帮助文档到输出页面里

def get_10pct_log_scale(last_price = 16.04):
    import math
    '''
    通达信的k线分析图里的对数坐标系10%等比坐标值的计算方法: 
    last_prince: 初始的y值, 右侧第二个k线的收盘价
    然后, 在此基础上向下2个板, 向上四个板画线的价格
    '''
    
    y= [ last_price * math.pow(1.1, i ) for i in range(-2,5)]
    print y

In [28]:
get_10pct_log_scale(12)

[9.917355371900825, 10.909090909090908, 12.0, 13.200000000000001, 14.520000000000003, 15.972000000000005, 17.569200000000006]


In [20]:
#print xrange(5)
#print math.exp(2)
#math.expm1?
#math.pow?

xrange(5)


'''
在talib包里有一个func.pyd文件(python dll文件), 他就是python版本的talib.dll
可以通过import语句来导入该模块, 然后通过dir来查看里面的接口函数
func.pyd
'''
from talib import func
dir(func)

talib.get_functions()

In [ ]:
adc

In [ ]:
asasx

In [ ]:
asasx